In [1]:
%load_ext autoreload
%autoreload 2

import time

import numpy as np
import torch
from torch.nn import functional as F
from torch import optim as torch_opt

from flare import trainer
from flare.callbacks import Checkpoint

from models.mnist import ConvNet

In [2]:
def _ceil(x):
    return int(np.round(x))

callno = 0
scoreboard = dict()
def make_target_fn(model_prefix, _device, model_class, trn_gen, val_gen, n_epochs, image_sz, n_classes, hyperparams):
    def target_fn(hyperparam_values):
        global callno
        global scoreboard

        # Ensuring that hyperparams is a 1D-tensor
        hyperparam_values = np.asarray(hyperparam_values).ravel()

        model_hyperparams = {hname: _ceil(hvalue) for hname, hvalue in zip(hyperparams, hyperparam_values)}
        model = model_class(image_sz, n_classes, **model_hyperparams)
        model = model.to(_device)
        loss_fn = F.nll_loss

        # The last hyperparam is the learning rate. We could use momentum, whatever as well
        nn_optimizer = torch_opt.SGD(model.parameters(), lr=hyperparam_values[-1], momentum=hyperparam_values[-2])

        filename = '{}_{}'.format(model_prefix, callno)
        cbs = [Checkpoint('val_accuracy', min_delta=1e-3, filename=filename, save_best=True, increasing=True)]

        # Training
        history = trainer.train_on_loader(model, trn_gen, val_gen, loss_fn, nn_optimizer,
                                          n_epochs=n_epochs, batch_first=True, device=_device,
                                          callbacks=cbs)
        
        # Getting the best model during training to evaluate
        best_model = torch.load(filename + '.pth').to(_device)
        eval_metrics = trainer.evaluate_on_loader(best_model, val_gen, loss_fn, batch_first=True,
                                                  device=_device, verbosity=0)
        
        # Opytimizer minimizes functions
        fitness = 1 - eval_metrics['accuracy']
        scoreboard[callno] = fitness
        callno += 1        
        return fitness
    return target_fn

In [3]:
device = torch.device('cpu')
pin_memory = False
if torch.cuda.is_available():
    device = torch.device('cuda')
    pin_memory = True
    torch.backends.cudnn.benchmark = True

In [38]:
import torchvision
BATCH_SIZE = 128

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=BATCH_SIZE, shuffle=True, pin_memory=pin_memory)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=BATCH_SIZE, shuffle=False, pin_memory=pin_memory)

In [5]:
from opytimizer.optimizers.fa import FA
from models import utils

target_fn = make_target_fn('./trained/firefly',
                           device,
                           ConvNet,
                           train_loader,
                           test_loader,
                           n_epochs=3,
                           image_sz=28,
                           n_classes=10,
                           hyperparams=['filters_1'])

N_AGENTS = 2
N_ITERATIONS = 2
# filter_1, lr, momentum
lower_bound = [1, 1e-3, 0]
upper_bound = [20, 1e-2, 1]

assert len(lower_bound) == len(upper_bound)

n_variables = len(lower_bound)
meta_hyperparams = dict(alpha=0.5, beta=0.2, gamma=1.0)

start = time.time()
history = utils.optimize(FA,
                         target=target_fn,
                         n_agents=N_AGENTS,
                         n_variables=n_variables,
                         n_iterations=N_ITERATIONS,
                         lb=lower_bound,
                         ub=upper_bound,
                         hyperparams=meta_hyperparams)
end = time.time() - start
print(end)

2019-08-02 10:05:05,513 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2019-08-02 10:05:05,514 - opytimizer.core.space — DEBUG — Running private method: build().
2019-08-02 10:05:05,515 - opytimizer.core.space — DEBUG — Running private method: check_bound_size().
2019-08-02 10:05:05,515 - opytimizer.core.space — DEBUG — Bound checked.
2019-08-02 10:05:05,516 - opytimizer.core.space — DEBUG — Running private method: check_bound_size().
2019-08-02 10:05:05,517 - opytimizer.core.space — DEBUG — Bound checked.
2019-08-02 10:05:05,518 - opytimizer.core.space — DEBUG — Running private method: create_agents().
2019-08-02 10:05:05,518 - opytimizer.core.space — DEBUG — Agents: 2 | Size: (3, 1) | Iterations: 2 | Lower Bound: [1, 0.001, 0] | Upper Bound: [20, 0.01, 1] | Built: True.
2019-08-02 10:05:05,519 - opytimizer.spaces.search — DEBUG — Running private method: initialize_agents().
2019-08-02 10:05:05,520 - opytimizer.spaces.search — DEBUG — Agents initialized.
2


2019-08-02 10:07:12,398 - opytimizer.optimizers.fa — INFO — Iteration 1/2



2019-08-02 10:09:19,837 - opytimizer.optimizers.fa — INFO — Fitness: 0.021100000000000008
2019-08-02 10:09:19,838 - opytimizer.optimizers.fa — INFO — Position: [[1.02040917e+01]
 [7.02968199e-03]
 [4.35869458e-01]]
2019-08-02 10:09:19,839 - opytimizer.optimizers.fa — INFO — Iteration 2/2



2019-08-02 10:11:24,709 - opytimizer.optimizers.fa — INFO — Fitness: 0.021100000000000008
2019-08-02 10:11:24,710 - opytimizer.optimizers.fa — INFO — Position: [[1.02040917e+01]
 [7.02968199e-03]
 [4.35869458e-01]]
2019-08-02 10:11:24,710 - opytimizer.opytimizer — INFO — Optimization task ended.
2019-08-02 10:11:24,712 - opytimizer.opytimizer — INFO — It took 379.1794002056122 seconds.
379.19931197166443


In [21]:
top_indices, top_fitness = utils.get_top_models(scoreboard, 2)
for ti, tf in zip(top_indices, top_fitness):
    print(f'{ti:<5} {tf:5}')


best_models = utils.load_models('./trained', 'firefly', top_indices)
for idx, model in zip(top_indices, best_models):
    utils.predict_persist(model, test_loader, device, f'predictions/firefly_{idx}.txt')

utils.store_labels(test_loader, './predictions/labels.txt')